## 21. Capturing and inspecting HTTP traffic

Debugging APIs is easier when you **see** the raw wire data.

* **Browser dev‑tools → Network tab** – inspect headers, status, CORS issues.
* **mitmproxy / Charles** – act as man‑in‑the‑middle, record and replay requests; useful for mobile apps.
* **curl ‑‑trace / httpie ‑‑print=HBhb** – CLI alternatives.

Remember to trust the mitmproxy root certificate on client if you inspect HTTPS.

```bash
mitmproxy
# then configure your app to use HTTP proxy 127.0.0.1:8080
```

### Quick check

1. True / False TLS must be terminated to read HTTPS payloads.

2. `curl -v` shows:
  a. only response  b. request + response headers

<details><summary>Answer key</summary>

1. **True** – proxy re‑encrypts after inspection.
2. **b**.

</details>

## 22. Reproducing race conditions

Concurrency bugs vanish when debugged. Strategies:
* **Stress loops** – run failing test 10 000×: `pytest -q --count=1000` (pytest‑repeat).
* **Random hash seed** – `PYTHONHASHSEED=random` surfaces order‑dependency.
* **pytest‑xdist** ‑n auto – parallel workers amplify races.
* Sleep jitter or `threading.Barrier` to force interleaving.

```bash
PYTHONHASHSEED=random pytest tests/test_race.py --count=500
```

### Quick check

1. `PYTHONHASHSEED` affects:
  a. list order  b. dict/hash order

2. True / False Running tests in parallel can expose shared‑state bugs.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>

## 23. Continuous Integration best practices

CI turns your test suite into a gatekeeper.

* **Fail‑fast**: stop pipeline on first red step → faster feedback.
* **Test sharding**: distribute slow suites (`pytest -n`) or GH Actions matrix.
* **Cache & artifacts**: save `pip` cache, upload coverage HTML, failed screenshots.
* Pin tool versions to avoid “works‑on‑my‑machine” surprises.

```yaml
name: ci
on: [push]
jobs:
  tests:
    runs-on: ubuntu-latest
    strategy:
      matrix: python: [3.10, 3.11]
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with: {python-version: ${{ matrix.python }}}
      - run: pip install -r req.txt
      - run: pytest -q
```

### Quick check

1. Uploading coverage HTML is an example of:
  a. test sharding  b. artifact retention

2. True / False Running the slowest tests first improves fail‑fast feedback.

<details><summary>Answer key</summary>

1. **b**.
2. **True** – pipeline fails earlier.

</details>

## 24. Test data management

* **Factories** (factory‑boy) build objects with defaults; flexible and DRY.
* **Static fixtures** – YAML / JSON seed files; fast to load but harder to vary.
* **Parameterized builders** – combine factories + fixture scopes.

Choose per layer: factories for unit tests, small seed DB for integration.

```python
import factory
class User:
    def __init__(self, name, active=True): self.name, self.active = name, active
class UserFactory(factory.Factory):
    class Meta: model = User
    name = factory.Faker('name')
```

### Quick check

1. Seed file data is loaded:
  a. dynamically per test  b. once and reused

2. True / False Factories help avoid brittle hard‑coded IDs.

<details><summary>Answer key</summary>

1. **b** – usually once per DB.
2. **True**.

</details>

## 25. When to delete tests

Tests are code with maintenance cost. Delete when:
* They duplicate coverage of higher‑level tests.
* Assert behaviour that was intentionally changed.
* Flaky due to low value (e.g., UI pixel tests).

Before deletion, ensure alternative coverage exists and communicate in PR.

```text
Scenario: API v1 sunset — remove v1 tests after deprecation window.
```

### Quick check

1. Keeping obsolete tests can:
  a. speed up CI  b. block refactors

2. True / False Deleting tests requires updating coverage expectations.

<details><summary>Answer key</summary>

1. **b**.
2. **True**.

</details>